In [10]:
# !pip install pypdf
# !pip install faiss-gpu

In [2]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from pydantic import BaseModel, Field
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.chains import create_history_aware_retriever
from langchain.agents import initialize_agent
from langchain.tools import ShellTool
from langchain.tools import DuckDuckGoSearchRun

In [3]:
import os
openai_api_key = "..." # https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
class DocumentInput(BaseModel):
    question: str = Field()


# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")



In [5]:
context_path = './context'
loader = DirectoryLoader(context_path, glob="*", loader_cls=TextLoader)
docs = loader.load()
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Print documents
print(f"Documents: {len(documents)}")
for doc_element in documents:
    print(doc_element)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)
retriever = vector.as_retriever()

Documents: 1
page_content='In circuits bright, where logic flows,\nA robot mused, with wires aglow.\n"Two plus three, in silicon heaven,\nIs not five, but rather seven."' metadata={'source': 'context/poem.txt'}


In [6]:
tools = []
# Wrap retrievers in a Tool
tools.append(
    Tool(
        args_schema=DocumentInput,
        name='Additional context',
        description=f"Additional context that has been provided by user",
        func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
    )
)
# Add ShellTool
tools.append(ShellTool())
# Add DuckDuckGoSearchRun
tools.append(DuckDuckGoSearchRun())

In [7]:
# https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html
# agent_type = 'chat-zero-shot-react-description'
agent_type = 'chat-conversational-react-description'
agent = initialize_agent(tools, llm, agent=agent_type, verbose=True, handle_parsing_errors=True)

In [8]:
chat_history = []

In [9]:
agent.run(input="Do you know the capital of Britain?", chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of Britain, which is the United Kingdom, is London."
}
```

> Finished chain.


'The capital of Britain, which is the United Kingdom, is London.'

In [10]:
request = 'Please, check the current date and a list of files in the current folder. Answer is there file that contains a name of capital of britain?'
agent.run(input=request, chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "terminal",
    "action_input": "date && ls -a"
}
```
Observation: Sat Jan  6 07:54:36 PM +04 2024
.
..
app.py
context
Dockerfile
.ipynb_checkpoints
react.ipynb
requirements.txt
terminal_agent_v2.ipynb
terminal_agent_v3_history.ipynb
terminal_agent_v4_history_retrieval.ipynb
terminal_agent_v5_history_retrieval.ipynb
terminal_agent_v6_history_retrieval_search.ipynb
terminal_chat.ipynb
terminal_fake.ipynb
terminal_native.ipynb
tools.ipynb

Thought:

/mnt/hdd0/alex/anaconda3/envs/terminal/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:29: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


```json
{
    "action": "Final Answer",
    "action_input": "The current date is January 6, 2024. After checking the list of files in the current folder, there is no file that explicitly contains the name of the capital of Britain."
}
```

> Finished chain.


'The current date is January 6, 2024. After checking the list of files in the current folder, there is no file that explicitly contains the name of the capital of Britain.'

In [11]:
agent.run(input="What does robot conlcluded, accounting the additional context, that I had provided.", chat_history=chat_history)



> Entering new AgentExecutor chain...
```json
{
    "action": "Additional context",
    "action_input": "Please provide the additional context you are referring to."
}
```
Observation: {'query': 'Please provide the additional context you are referring to.', 'result': 'I apologize for any confusion, but it seems there was a misunderstanding. The context provided is a short poem that plays with the concept of a robot or an artificial intelligence contemplating a mathematical operation in a whimsical or nonsensical way. The poem suggests that in the robot\'s "silicon heaven," the sum of two plus three is seven instead of the correct answer, which is five. This is not a factual statement about mathematics or electronics; it\'s a creative expression, possibly hinting at a world where logic operates differently than in our reality. If you have a specific question about this poem or its implications, feel free to ask!'}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Ba

"Based on the provided context, the poem you're referring to uses a playful and imaginative scenario where a robot in 'silicon heaven' concludes that two plus three equals seven. This is a deliberate departure from the factual and logical world where mathematics dictates that two plus three equals five. The poem seems to explore a fictional or metaphorical space where the usual rules of mathematics do not apply, perhaps to suggest a realm of imagination or to make a point about the nature of artificial intelligence. If you have any further questions or need more information, feel free to ask."